In [1]:
import time
import os
import openai
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2
from io import BytesIO

In [3]:
#client = OpenAI()

In [4]:
class LMFunction(object):
    def __init__(self, engine='gpt-4', max_tokens=512):
        self.engine = engine
        self.max_tokens = max_tokens
        self.client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def _call_api(self, prompt, engine, max_tokens, max_retries=10, retry_wait=2):
        for i in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=engine,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=max_tokens,
                    temperature=1.0
                )
                return response
            except openai.APIError as e:  # New error handling
                time.sleep(retry_wait)
        return {'choices': [{'message': {'content': ''}}]}

    def _parse_message(self, msg):
        try:
            content = msg.choices[0].message.content
        except (IndexError, AttributeError):
            content = ''
        return content

    def f(self, prompt, x):
        msg = self._call_api(
            prompt=prompt+x,
            engine=self.engine,
            max_tokens=self.max_tokens
        )
        evaluation = self._parse_message(msg)
        return evaluation


class Checker(object):
    """A modified version of the Draft, Sketch, Prove proof-checking client.
    (https://github.com/albertqjiang/draft_sketch_prove/blob/main/autoformalization/checker.py)

    This checker supports Isabelle2022 via the new version of PISA
    (https://albertqjiang.github.io/Portal-to-ISAbelle/).

    It supports checking a miniF2F-style proof via `check`.

    Finally, it replaces `sledgehammer` with a call to `normalhammer`.
    """
    def __init__(self, working_dir, isa_path, theory_file, port=9000):
        sys.path.append(os.environ['PISA_PATH'])
        try:
            from pisa_client import initialise_env
            self.initialise_env = initialise_env
        except:
            print("Set $PISA_PATH to /yourpath/to/Portal-to-ISAbelle/src/main/python")

        self.working_dir = working_dir
        self.isa_path = isa_path
        self.theory_file = theory_file
        self.port = port

    def _initialize(self):
        env = self.initialise_env(
            self.port,
            isa_path=self.isa_path,
            theory_file_path=self.theory_file,
            working_directory=self.working_dir
        )
        return env

    def _exit(self, env):
        try:
            env.post('exit')
        except:
            print("env.post('exit') timed out")
            pass
        os.system("ps aux | grep Isabelle | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")
        os.system("ps aux | grep poly | awk '{print $2}' | xargs kill -9 > /dev/null 2>&1")

    def _parse_output(self, obs):
        """Parse the sledgehammer output, otherwise return an empty string"""
        if '<hammer>' in obs:
            output = obs.split('<hammer>')[0]
        else:
            output = ''
        return output

    def _run_step(self, step, i, tls_name, env):
        obs, reward, done, metadata = env.step_to_top_level_state(
            action=step,
            tls_name=tls_name,
            new_name='default_%d' % i
        )
        error = None
        if 'error:' in obs or 'Step error' in obs or 'Unknown error' in obs:
            error = obs
        return obs, reward, done, metadata, error

    def _run_sledgehammer(self, step, i, tls_name, env):
        # First try heuristics
        for heuristic in ['by auto', 'by simp', 'by blast', 'by fastforce', 'by force', 'by eval', 'by presburger', 'by sos', 'by arith', 'by linarith', 'by (auto simp: field_simps)']:
            step_ = step.replace('normalhammer', heuristic)
            obs, reward, done, metadata, error = self._run_step(step_, i, tls_name, env)
            if error is None:
                obs = '%s <hammer> %s' % (heuristic, obs)
                return obs, reward, done, metadata, error
        # Try sledgehammer
        out = self._run_step(step, i, tls_name, env)
        return out

    def check(self, statement_and_proof):
        # Initialize environment
        env = self._initialize()
        env.initialise()

        # Wrap and parse theorem
        theory = Checker.wrap_theorem(statement_and_proof)
        steps = Checker.get_parsed(env, theory)

        result = self._check(env, steps)
        return result

    def _check(self, env, steps):
        done = False
        reason = ''
        success = False
        step_results = []
        tls_name = 'default'
        for i, step in enumerate(steps):
            try:
                time0 = time.time()
                if 'normalhammer' in step:
                    obs, reward, done, metadata, error = self._run_sledgehammer(step, i, tls_name, env)
                else:
                    obs, reward, done, metadata, error = self._run_step(step, i, tls_name, env)
                step_time = time.time() - time0
                step_results.append(dict(index=i, step=step, output=self._parse_output(obs), step_time=step_time))
                if error is not None:
                    reason = error
                    success = False
                    done = False
                    break
            except:
                # Timeout - end the proof attempt
                success = False
                done = False
                reason = 'timeout (%d)' % len(step_results)
                step_results.append(dict(index=i, step=step, output=''))
                break

            # Change when successful
            tls_name = 'default_%d' % i

        if done and reward == 1.0:
            success = True

        result = {
            'success': success,
            'reason': reason,
            'num_steps': len(steps),
            'last_step': len(step_results),
            'step_results': step_results,
            'theorem_and_proof': self.reconstruct(step_results) if success else ''
        }
        # Exit environment
        self._exit(env)
        return result
    
    @staticmethod
    def reconstruct(step_results):
        steps = []
        for step_result in step_results[1:]:
            if step_result['output'] != '':
                steps.append(step_result['output'].strip())
            else:
                steps.append(step_result['step'].strip())
        theorem_and_proof = '\n'.join(steps)
        return theorem_and_proof

    @staticmethod
    def wrap_theorem(theorem):
        return 'theory Interactive imports HOL.HOL Complex_Main "HOL-Library.Code_Target_Numeral" "HOL-Library.Sum_of_Squares" "Symmetric_Polynomials.Vieta" "HOL-Computational_Algebra.Computational_Algebra" "HOL-Number_Theory.Number_Theory" \n begin\n%s' % theorem

    @staticmethod
    def get_parsed(env, theory, tls_name='default'):
        # HACK: the parsing doesn't work well with `normalhammer`, so we replace
        # all hammer calls with sorry, then replace sorry to normalhammer after parsing.
        theory = theory.replace('sledgehammer', 'sorry')
        theory = theory.replace('normalhammer', 'sorry')

        steps = env.post(f"<parse text> ${theory}")
        steps = steps.split('<SEP>')
        steps = [s for s in steps if s.strip() != '']
        # remove weird '$' step and whitespace steps
        steps = [s for s in steps if s != '$' and s.strip() != '']
        steps = [s.replace('sorry', 'normalhammer') for s in steps]
        return steps


In [5]:
import sys
import os
sys.path.append('../')
os.environ['PISA_PATH'] = '/home/balaji/Desktop/Theorem_Proving/Portal-to-ISAbelle/src/main/python'

checker = Checker(
    working_dir='/home/balaji/Desktop/Theorem_Proving/Isabelle2022/src/HOL/Examples',
    isa_path='/home/balaji/Desktop/Theorem_Proving/Isabelle2022',
    theory_file='/home/balaji/Desktop/Theorem_Proving/Isabelle2022/src/HOL/Examples/Interactive.thy',
    port=9000
)

In [6]:
theorem_and_sledgehammer_proof = """

locale graph = fixes G :: "'a list" (* G is a list of edges, where each edge is a pair of vertices *) fixes tail :: "'a ⇒ 'b" (* Function to get the tail of an edge *) fixes head :: "'a ⇒ 'b" (* Function to get the head of an edge *) assumes non_empty_path: "length p > 0" assumes u_tail: "u = tail (hd p)" assumes v_head: "v = head (last p)" assumes path_continuity: "∀i. Suc i < length p ⟶ head (G ! i) = tail (G ! (i + 1))" definition directed_path :: "'b ⇒ 'b ⇒ 'a list ⇒ bool" where "directed_path u v p ⟷ (∃path. path ≠ [] ∧ hd path = u ∧ last path = v ∧ (∀i. Suc i < length path ⟶ head (path ! i) = tail (path ! (i + 1))))" lemma (in graph) directed_path_exists: assumes "p ≠ []" "u = tail (hd p)" "v = head (last p)" "∀i. Suc i < length p ⟶ head (G ! i) = tail (G ! (i + 1))" shows "directed_path u v p" proof - have "∃path. path = p ∧ path ≠ [] ∧ hd path = u ∧ last path = v ∧ (∀i. Suc i < length path ⟶ head (path ! i) = tail (path ! (i + 1)))" proof show "p = p" by simp show "p ≠ []" using assms(1) by simp show "hd p = u" using assms(2) by simp show "last p = v" using assms(3) by simp show "∀i. Suc i < length p ⟶ head (G ! i) = tail (G ! (i + 1))" using assms(4) by simp qed thus ?thesis using directed_path_def by auto qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])




==== Success: False
--- Complete proof:



In [ ]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm 

ds = load_dataset("kings-crown/FVELER_Isabelle")
df = pd.DataFrame(ds["train"]) 

if "proof" not in df.columns:
    raise ValueError("Column 'proof' not found in the dataset")


def check_proof(proof):
    result = checker.check(proof)  
    return result.get("success", False)  

tqdm.pandas(desc="Processing proofs")  # Initialize tqdm for Pandas
df["success"] = df["proof"].progress_apply(check_proof)
updated_ds = ds["train"].add_column("success", df["success"].tolist())

updated_ds.to_csv("successful_dataset.csv")

print("Processing complete.")


README.md:   0%|          | 0.00/630 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/52.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26192 [00:00<?, ? examples/s]

Processing proofs:   0%|                  | 18/26192 [01:21<34:37:21,  4.76s/it]

In [7]:
theorem_and_sledgehammer_proof = """

lemma sbintrunc_Suc_numeral: "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) 1 = 1" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (numeral (Num.Bit0 w)) = 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (numeral (Num.Bit1 w)) = 1 + 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (- numeral (Num.Bit0 w)) = 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (- numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (- numeral (Num.Bit1 w)) = 1 + 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (- numeral (w + Num.One))" by (simp_all add: signed_take_bit_Suc)
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])



==== Success: True
--- Complete proof:
lemma sbintrunc_Suc_numeral: "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) 1 = 1" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (numeral (Num.Bit0 w)) = 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (numeral (Num.Bit1 w)) = 1 + 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (- numeral (Num.Bit0 w)) = 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (- numeral w)" "(signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) (Suc n) (- numeral (Num.Bit1 w)) = 1 + 2 * (signed_take_bit :: nat \<Rightarrow> int \<Rightarrow> int) n (- numeral (w + Num.One))"
by (simp_all add: signed_take_bit_Suc)


In [9]:
theorem_and_sledgehammer_proof = """

lemma always_weakly_fair: shows "⟦ weakly_fair enabled taken ⟧" proof - (* Use sledgehammer to find a proof strategy *) have "⟦ weakly_fair enabled taken ⟧" by sledgehammer thus ?thesis by simp qed

"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])


==== Success: False
--- Complete proof:



In [ ]:
import json
examples = [
    {"tag": "aimeI_2000_p7", "category": "algebra", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nSuppose that $x,$ $y,$ and $z$ are three positive numbers that satisfy the equations $xyz = 1,$ $x + \\frac {1}{z} = 5,$ and $y + \\frac {1}{x} = 29.$ Then $z + \\frac {1}{y} = \\frac {m}{n},$ where $m$ and $n$ are [[relatively prime]] positive integers. Find $m + n$. Show that it is 5.\n\n\nnote: this is the type of problem that makes you think symmetry, but actually can be solved easily with substitution, and other normal technniques\n\n### Solution\n\nWe can rewrite $xyz=1$ as $\\frac{1}{z}=xy$.\n\nSubstituting into one of the given equations, we have \n$x+xy=5$\n$x(1+y)=5$\n$\\frac{1}{x}=\\frac{1+y}{5}.$\n\nWe can substitute back into $y+\\frac{1}{x}=29$ to obtain\n$y+\\frac{1+y}{5}=29$\n$5y+1+y=145$\n$y=24.$\n\nWe can then substitute once again to get\n$x=\\frac15$\n$z=\\frac{5}{24}.$\nThus, $z+\\frac1y=\\frac{5}{24}+\\frac{1}{24}=\\frac{1}{4}$, so $m+n=005$.*)\n\nFormal:\ntheorem\n  fixes x y z :: real\n    and p :: rat\n  assumes \"0 < x \\<and> 0 < y \\<and> 0 < z\"\n    and \"x * y * z = 1\"\n    and \"x + 1 / z = 5\"\n    and \"y + 1 / x = 29\"\n    and \"z + 1 / y = p\"\n    and \"0 < p\" \n  shows \"let (m,n) = quotient_of p in m + n = 5\"\nproof -\n  (* We can rewrite $xyz=1$ as $\\frac{1}{z}=xy$. *)\n  have c0: \"z = 1 / (x*y)\"\n    sledgehammer\n  (* Substituting into one of the given equations, we have \n  $x+xy=5$\n  $x(1+y)=5$\n  $\\frac{1}{x}=\\frac{1+y}{5}.$ *)\n  have c1: \"1 / x = (1+y) / 5\" \n  proof -\n    have \"x + x * y = 5\" using assms(3) unfolding c0\n      sledgehammer\n    then have \"x * (1 + y) = 5\"\n      sledgehammer\n    then have t1: \"x = 5 / (1+y)\"\n      sledgehammer\n    then show ?thesis\n      sledgehammer\n  qed\n  (* We can substitute back into $y+\\frac{1}{x}=29$ to obtain\n  $y+\\frac{1+y}{5}=29$\n  $5y+1+y=145$\n  $y=24.$ *)\n  have \"y + (1+y)/5 = 29\" using assms(4) unfolding c1 sledgehammer\n  then have \"5* (y + (1+y)/5) = 5 * 29\" sledgehammer\n  also have \"... = 145\" sledgehammer\n  finally have c2_1: \"5* (y + (1+y)/5) = 145\" sledgehammer\n  have \"5* (y + (1+y)/5) = 5*y + (1+y)\" sledgehammer\n  also have \"... = 6*y + 1\" sledgehammer\n  finally have c2_2: \"5* (y + (1+y)/5) = 6*y + 1\" sledgehammer\n  have \"6*y + 1 = 145\" using c2_1 c2_2 sledgehammer\n  then have c2: \"y = 24\" sledgehammer\n  (* We can then substitute once again to get\n  $x=\\frac15$\n  $z=\\frac{5}{24}.$ *)\n  have \"1/x = 5\" using c1 unfolding c2 sledgehammer\n  then have c3: \"x = 1/5\"\n    sledgehammer\n  then have c4: \"z = 5/24\"\n    sledgehammer\n  (* Thus, $z+\\frac1y=\\frac{5}{24}+\\frac{1}{24}=\\frac{1}{4}$, so $m+n=005$. *)\n  have \"p = z + 1/y\" using assms(5) sledgehammer\n  also have \"... = 5/24 + 1/24\" unfolding c2 c4 sledgehammer\n  also have \"... = 1/4\" sledgehammer\n  finally have c5: \"p = 1/4\"\n    sledgehammer\n  have \"quotient_of p = (1, 4)\" unfolding c5 sledgehammer\n  then show ?thesis sledgehammer\nqed"},
    {"tag": "algebra_2rootsintpoly_am10tap11eqasqpam110", "category": "algebra", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nShow that for any complex number a, $(a-10)(a+11) = a^2 + a - 110$.\n\n### Solution\n\nWe first expand all terms of the left hand side to get $a^2 - 10a + 11a - 10*11$.\nThis equals $a^2 + a - 10*11 = a^2 + a - 110$.*)\n\nFormal:\ntheorem\n  fixes a :: complex\n  shows \"(a-10) * (a+11) = a^2 + a -110\"\nproof -\n  (* We first expand all terms of the left hand side to get $a^2 - 10a + 11a - 10*11$. *)\n  have \"(a-10) * (a+11) = a^2 - 10*a + 11*a - 10 *11\"\n    sledgehammer\n  (* This equals $a^2 + a - 10*11 = a^2 + a - 110$. *)\n  also have \"\\<dots> = a^2 + a - 10 * 11\"\n    sledgehammer\n  also have \"\\<dots> = a^2 + a - 110\"\n    sledgehammer\n  finally show ?thesis\n    sledgehammer\nqed"},
    {"tag": "mathd_numbertheory_335", "category": "number_theory", "metadata": {}, "prompt": "Informal:\n(*### Problem\n\nWhen Rachel divides her favorite number by 7, she gets a remainder of 5. What will the remainder be if she multiplies her favorite number by 5 and then divides by 7? Show that it is 4.\n\n### Solution\n\nLet $n$ be Rachel's favorite number. \nThen $n \\equiv 5 \\pmod{7}$, so $5n \\equiv 5 \\cdot 5 \\equiv 25 \\equiv 4 \\pmod{7}$.\n*)\n\nFormal:\ntheorem\n  fixes n :: nat\n  assumes h0 : \"n mod 7 = 5\"\n  shows \"(5 * n) mod 7 = 4\"\nproof -\n  (* Then $n \\equiv 5 \\pmod{7}$, so $5n \\equiv 5 \\cdot 5 \\equiv 25 \\equiv 4 \\pmod{7}$. *)\n  have c0:\"(5 * n) mod 7 = (5 * 5) mod 7\" using h0\n    sledgehammer\n  then have \"\\<dots> = 4\" sledgehammer\n  then have \"(5 * n) mod 7 = 4\" using c0 sledgehammer\n  then show ?thesis sledgehammer\nqed"}
]

In [ ]:
prompt = """Draft an informal solution similar to below. 
The informal solution will be used to sketch a formal Isabelle proof.
Here are some examples:
"""
for x in examples:
    prompt += ("Example:\n" + x['prompt'][:x['prompt'].find('Formal:')] + "\n\n")
prompt += """Informal:
(*### Problem

"""

print(prompt)

In [ ]:
p = LMFunction('gpt-4o')
xi = 'Statement in natural language: The lemma named "always_weakly_fair" shows that "weakly_fair enabled taken" is always true.'
yi = p.f(prompt, xi)
print(yi)

In [ ]:
prompt = """Translate the informal solution into a sketch of the
formal Isabelle proof. Add `sledgehammer` in the sketch whenever
possible. `sledgehammer` will be used to call the automated Sledgehammer prover. 
"""
for x in examples:
    prompt += (x['prompt'] + "\n\n")
prompt += """Informal:
(*### Problem

"""

zi = p.f(prompt, xi + '\n\n' + yi )
print(zi)







In [ ]:
theorem_and_sledgehammer_proof = """
lemma always_weakly_fair:
  assumes wf: "weakly_fair (enabled taken)"
  shows "always (\<lambda>s. enabled taken s \<longrightarrow> eventually (\<lambda>s. taken s) s)"
proof -
  (* 1. Start by recalling the definition of "weakly_fair": *)
  have wf_def: "weakly_fair P \<longleftrightarrow> (\<forall>s. (\<forall>t. eventually P t) \<longrightarrow> eventually Q t)"
    sorry  (* Replace with actual definition in your context *)

  (* 2. Understand when "enabled" conditions apply: *)
  have enabled_def: "enabled a = {s. s enables a}"
    sorry  (* Replace with actual definition in your context *)

  (* 3. "always_weakly_fair" means whenever "enabled" holds, "taken" must eventually occur *)
  {
    fix s
    assume "enabled taken s"

    (* 4-5. Suppose "enabled" is met infinitely often, "weakly_fair" implies "taken" follows *)
    have evt_taken: "\<forall>t. eventually (enabled taken) t \<longrightarrow> eventually (taken) t"
      using wf wf_def by blast  (* Using the assumption of weakly fair *)

    hence "eventually (taken) s" using evt_taken `enabled taken s` by blast
  }
  hence "\<forall>s. enabled taken s \<longrightarrow> eventually taken s" by blast

  (* 6. Thus, "always" condition holds *)
  thus ?thesis using eventually_def by auto
qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])

In [13]:
theorem_and_sledgehammer_proof = """
theorem
  fixes x :: int
  assumes h0: "even x"
  shows "odd (x + 5)"
proof -
  (* We start by recalling the definition of an even number: a number is even if it can be expressed as \( x = 2k \), where \( k \) is an integer. 
     Given that \( x \) is even, we can write \( x = 2k \). *)
  obtain k where x_def: "x = 2 * k"
    using h0 by auto
  (* To find \( x+5 \), we substitute for \( x \): \( x + 5 = 2k + 5. \) *)
  have "x + 5 = 2 * k + 5"
    by (simp add: x_def)
  (* Now, let's consider the parity (evenness or oddness) of \( x+5 \). We can rewrite \( x+5 \) as \( x + 5 = 2k + 5 = 2k + 4 + 1 = 2(k + 2) + 1. \) *)
  have "(2 * k + 5) = 2 * (k + 2) + 1"
    by auto
  (* Since \( 2(k + 2) \) is clearly even, adding 1 makes the entire expression \( 2(k + 2) + 1 \) odd. *)
  have "odd (2 * (k + 2) + 1)"
    by simp
  (* Finally, since the expression is odd, we conclude that if \( x \) is even, then \( x + 5 \) is odd. *)
  then show ?thesis
    by (simp add: x_def)
qed
"""

result = checker.check(theorem_and_sledgehammer_proof)

print("\n==== Success: %s" % result['success'])
print("--- Complete proof:\n%s" % result['theorem_and_proof'])


==== Success: True
--- Complete proof:
theorem
  fixes x :: int
  assumes h0: "even x"
  shows "odd (x + 5)"
proof -
(* We start by recalling the definition of an even number: a number is even if it can be expressed as \( x = 2k \), where \( k \) is an integer. 
     Given that \( x \) is even, we can write \( x = 2k \). *)
obtain k where x_def: "x = 2 * k"
using h0
by auto
(* To find \( x+5 \), we substitute for \( x \): \( x + 5 = 2k + 5. \) *)
have "x + 5 = 2 * k + 5"
by (simp add: x_def)
(* Now, let's consider the parity (evenness or oddness) of \( x+5 \). We can rewrite \( x+5 \) as \( x + 5 = 2k + 5 = 2k + 4 + 1 = 2(k + 2) + 1. \) *)
have "(2 * k + 5) = 2 * (k + 2) + 1"
by auto
(* Since \( 2(k + 2) \) is clearly even, adding 1 makes the entire expression \( 2(k + 2) + 1 \) odd. *)
have "odd (2 * (k + 2) + 1)"
by simp
(* Finally, since the expression is odd, we conclude that if \( x \) is even, then \( x + 5 \) is odd. *)
then
show ?thesis
by (simp add: x_def)
qed
